In [1]:
import os
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
#from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.backend as K

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [4]:
BATCH_SIZE=32 * strategy.num_replicas_in_sync
MAX_LENGTH=270
PRE_TRAINED_NAME="bert-base-uncased"

In [ ]:
GCS_PATH = KaggleDatasets().get_gcs_path('shop-512-size-tfrecords')

In [5]:
df=pd.read_csv("processed_data/fold_data.csv")
df.head()

,posting_id,image,image_phash,title,label_group,image_path,gfold
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,666,train_images\0000a68812bc7e98c42888dfb1c07da0.jpg,0
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",7572,train_images\00039780dfc94d01db8676fe789ecd05.jpg,2
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,6172,train_images\000a190fdd715a2a36faed16e2c65df7.jpg,0
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,10509,train_images\00117e4fc239b1b641ff08340b429633.jpg,1
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,9425,train_images\00136d1cf4edede0203f32f05f660588.jpg,3


In [6]:
train_steps={}
valid_steps={}
for fold in range(5):
    valid_data=df.loc[df['gfold']==fold].reset_index(drop=True)
    valid_steps[fold]=valid_data.shape[0]
    train_steps[fold]=df.shape[0]-valid_data.shape[0]

In [7]:
from transformers import TFBertModel,TFRobertaModel,TFAlbertModel,TFXLNetModel
from transformers import BertTokenizer,RobertaTokenizer,AlbertTokenizer,XLNetTokenizer

In [8]:
TOKENIZER=BertTokenizer.from_pretrained("bert-base-uncased")

In [9]:
def MAKE_MODEL_INPUTS(data):
    INPUT_IDS=np.zeros((data.shape[0],MAX_LENGTH),dtype=np.int32)
    ATTENTION_MASK=np.zeros((data.shape[0],MAX_LENGTH),dtype=np.int32)
    TOKEN_TYPE_IDS=np.zeros((data.shape[0],MAX_LENGTH),dtype=np.int32)
    Y=np.zeros((data.shape[0],))
    n_iters=int(np.ceil(data.shape[0]/BATCH_SIZE))
    max_end=data.shape[0]
    for batch in tqdm(range(n_iters)):
        start=batch*BATCH_SIZE
        end=(batch+1)*BATCH_SIZE
        if end>max_end:
            end=max_end
        samples=data.iloc[start:end]['title'].values.tolist()
        tokenized_samples=TOKENIZER.batch_encode_plus(samples,max_length=MAX_LENGTH,padding='max_length',truncation='longest_first')
        INPUT_IDS[start:end,]=tokenized_samples['input_ids']
        ATTENTION_MASK[start:end,]=tokenized_samples['attention_mask']
        TOKEN_TYPE_IDS[start:end,]=tokenized_samples['token_type_ids']
        Y[start:end]=data.iloc[start:end]['label_group']
    MODEL_INPUTS=({"input_ids":INPUT_IDS,"attention_mask":ATTENTION_MASK,"token_type_ids":TOKEN_TYPE_IDS},Y)
    MODEL_OUTPUTS=Y
    return MODEL_INPUTS,MODEL_OUTPUTS

In [11]:
FOLDS_INPUTS={}
for i in range(5):
    train_data=df.loc[df['gfold']!=i].reset_index(drop=True)
    test_data=df.loc[df['gfold']==i].reset_index(drop=True)
    FOLDS_INPUTS[i]={}
    FOLDS_INPUTS[i]['train']=MAKE_MODEL_INPUTS(train_data)
    FOLDS_INPUTS[i]['test']=MAKE_MODEL_INPUTS(test_data)

100%|██████████| 215/215 [00:02<00:00, 82.13it/s]


In [13]:
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.models import Model

In [14]:
class ARCFACE_LAYER(tf.keras.layers.Layer):
    def __init__(self,m=0.5,s=30,n_classes=11014):
        super(ARCFACE_LAYER,self).__init__()
        self.m=m
        self.s=s
        self.sin_m=tf.sin(m)
        self.cos_m=tf.cos(m)
        self.n_classes=n_classes
        self.threshold = tf.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def build(self,input_shape):
        prev_layer_units=input_shape[0][1]
        self.w=self.add_weight(shape=(prev_layer_units,self.n_classes),trainable=True,
                              initializer='glorot_uniform')

    def get_config(self):
        config=super().get_config()
        config.update({"m":0.5,
                       "s":30,
                       "n_classes":11014})
        return config


    def call(self,inputs):
        prev_layer,y=inputs
        y=tf.cast(y,dtype=tf.int32)
        y_hot=tf.one_hot(y,self.n_classes)
        y_hot=tf.cast(y_hot,dtype=tf.float32)
        w_norm=tf.linalg.l2_normalize(self.w,axis=0)
        x_norm=tf.linalg.l2_normalize(prev_layer,axis=1)
        cos_theta=tf.linalg.matmul(x_norm,w_norm)
        sin_theta=tf.sqrt(1-tf.pow(cos_theta,tf.cast(2,dtype=tf.float32)))
        cos_theta_m=(cos_theta*self.cos_m)-(sin_theta*self.sin_m)
        cos_theta_m=tf.where(cos_theta>self.threshold,cos_theta_m,cos_theta-self.mm)
        final=self.s*((y_hot*cos_theta_m)+((1-y_hot)*cos_theta))
        return final

In [15]:
for fold in range(5):
    TRAIN_INPUTS,TRAIN_OUTPUTS=FOLDS_INPUTS[fold]['train']
    TEST_INPUTS,TEST_OUTPUTS=FOLDS_INPUTS[fold]['test']
    NUM_TRAIN_STEPS=train_steps[fold]//BATCH_SIZE
    NUM_VALID_STEPS=valid_steps[fold]//BATCH_SIZE
    tf.keras.backend.clear_session()
    with strategy.scope():
        input_ids=Input((MAX_LENGTH,),dtype=tf.int32)
        attention_mask=Input((MAX_LENGTH,),dtype=tf.int32)
        token_type_ids=Input((MAX_LENGTH,),dtype=tf.int32)
        ins=Input((),name="label_input")
        pre_trained=TFBertModel.from_pretrained(PRE_TRAINED_NAME,output_hidden_states=True)
        pre_outputs=pre_trained({"input_ids":input_ids,"attention_mask":attention_mask,
                        "token_type_ids":token_type_ids})
        hidden_layers=[]
        k=0
        for i in reversed(range(len(pre_outputs['hidden_states']))):
            if k<4:
                hidden_layers.append(pre_outputs['hidden_states'][i])
                k+=1
            else:
                break
        x=tf.keras.layers.Concatenate()(hidden_layers)[:,0,:]
        #x=pre_outputs['pooler_output']
        x=Dense(512)(x)
        arc_layer=ARCFACE_LAYER()
        x=arc_layer([x,ins])
        outs=tf.keras.layers.Softmax()(x)
        model=Model(inputs=({"input_ids":input_ids,"attention_mask":attention_mask,
                        "token_type_ids":token_type_ids},ins),outputs=outs)
        print(f"training for fold {fold}")
        if fold==0:
            print(model.summary())
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    early=tf.keras.callbacks.EarlyStopping(monitor="val_loss",mode="min",verbose=1,patience=10)
    saver=tf.keras.callbacks.ModelCheckpoint(filepath=PRE_TRAINED_NAME+f"{fold}.h5",
                                     monitor="val_loss",mode="min",save_best_only=True,
                                     save_weights_only=True,verbose=1)
    model.fit(TRAIN_INPUTS,TRAIN_OUTPUTS,
              validation_data=(TEST_INPUTS,TEST_OUTPUTS),epochs=40,
              callbacks=[early,saver],steps_per_epoch=NUM_TRAIN_STEPS,
             validation_steps=NUM_VALID_STEPS)
    print(f"model training for {fold} is done")
    del model
    import gc
    gc.collect()

Downloading: 100%|██████████| 511M/511M [01:53<00:00, 4.74MB/s]   
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


training for fold 0
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 270)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 270)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 270)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_2[0][0]',                
                                thPoolingAndCrossAt               'input_1

ResourceExhaustedError: Graph execution error:

Detected at node 'model/tf_bert_model/bert/encoder/layer_._0/intermediate/dense/Tensordot/MatMul' defined at (most recent call last):
    File "c:\Users\ravi1\anaconda3\envs\restaurant\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\ravi1\anaconda3\envs\restaurant\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\ravi1\anaconda3\envs\restaurant\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\ravi1\anaconda3\envs\restaurant\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\Users\ravi1\anaconda3\envs\restaurant\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\ravi1\anaconda3\envs\restaurant\lib\site-packages\tornado\platform\asyncio.py", line 199, in star